In [23]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
import torch
import torch.optim

from torch.autograd import Variable
from utils.common_utils import *
from torchvision import transforms

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
imsize=-1
dim_div_by = 64


NET_TYPE = 'skip_depth6' # one of skip_depth4|skip_depth2|UNET|ResNet



In [24]:
img_path  = 'data/in/wangfei.jpg'
mask_path = 'data/out/6_40_wangfei.jpg'
img_pil, img_np = get_image(img_path, imsize)
img_mask_pil, img_mask_np = get_image(mask_path, imsize)


In [60]:
pad = 'reflection'
OPT_OVER = 'net' # 'net,input'
reg_noise_std = 1./30. # set to 1./20. for sigma=50
LR = 0.01
OPTIMIZER='adam' # 'LBFGS'
show_every = 500
num_iter=2400
input_depth = 3
figsize = 5 
dtype = (torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor)

net = skip(
            input_depth, 1, 
            num_channels_down = [8, 16, 32, 64, 128], 
            num_channels_up   = [8, 16, 32, 64, 128],
            num_channels_skip = [4, 4, 4, 4, 4], 
            upsample_mode='bilinear',
            need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU')

net = net.type(dtype)
# print(net)

In [61]:
content_transform = transforms.Compose([
    transforms.RandomGrayscale(p=1),
    transforms.ToTensor()
])
timg = content_transform(img_pil)
print(timg.shape)
timg = timg.unsqueeze(0)
out = net(timg)
print(out.shape)

torch.Size([3, 995, 615])


/Users/roger/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1749: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


torch.Size([1, 1, 995, 615])


In [62]:
out = out.squeeze().unsqueeze(0).detach()
print(out.shape)
m = np_to_pil(out.numpy())
m.show()  #显示方法之一
# plt.imshow(np.asarray(m)) #显示方法之二

torch.Size([1, 995, 615])
